In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
def fileHandler (fileName):
    # load ascii text and covert to lowercase
    raw_text = open(fileName, 'r', encoding='utf-8').read()
    raw_text = raw_text.lower()
    # create mapping of unique chars to integers
    
    uniqueChars = sorted(list(set(raw_text)))
    vocabSize = len(uniqueChars)
    char_to_int = dict((c, i) for i, c in enumerate(uniqueChars))
    int_to_char = dict((i, c) for i, c in enumerate(uniqueChars))
    
    integers = [char_to_int[char] for char in raw_text]
    #We are doing One hot encoding as We want our MLP to work on it, and a classification problem
    dataMatrix = np.eye(vocabSize)[integers]
    return dataMatrix , vocabSize, char_to_int,int_to_char
    
    

### Text Generation with SIngle Input

In [3]:
dataMatrix , vocabSize, char_to_int,int_to_char = fileHandler("C:/Users/Lenovo/Desktop/My Git Repo/abcde.txt")

In [4]:
print(dataMatrix.shape,vocabSize,"\n",char_to_int,"\n",int_to_char)

(153594, 7) 7 
 {'\n': 0, ' ': 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6} 
 {0: '\n', 1: ' ', 2: 'a', 3: 'b', 4: 'c', 5: 'd', 6: 'e'}


#### Creating Training Data

In [5]:
# Prepare training input-output pairs
X_train = dataMatrix[:-1]  # All rows except the last one
Y_train = dataMatrix[1:]   # All rows except the first one

inputs = torch.Tensor(X_train)
#Take the max number (1 in our case) indice of the one hot enocding alongside column
labels = torch.Tensor(np.argmax(Y_train, axis=1)) 


In [6]:
print(int_to_char[2])
print(inputs[0])
print(int_to_char)
print()

a
tensor([0., 0., 1., 0., 0., 0., 0.])
{0: '\n', 1: ' ', 2: 'a', 3: 'b', 4: 'c', 5: 'd', 6: 'e'}



In [7]:
class TextGeneratorMLP(nn.Module):
    def __init__(self,inputSize,outputSize):
        super().__init__()
        
        self.InputLayer = nn.Linear(in_features=inputSize,out_features=128)
        self.HiddenLayer = nn.Linear(in_features=128,out_features=256)
        self.HiddenLayer2 = nn.Linear(in_features=256,out_features=64)
        self.OutputLayer = nn.Linear(in_features=64,out_features=outputSize)
    
    def forward(self,x):
        x = self.InputLayer(x)
        x = F.relu(x)
        
        x = self.HiddenLayer(x)
        x = F.relu(x)
        
        x = self.HiddenLayer2(x)
        x = F.relu(x)
        
        x = self.OutputLayer(x)
        
        return x
        

In [8]:
model  = TextGeneratorMLP(vocabSize,vocabSize)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    
    optimizer.zero_grad()
    outputs = model(inputs)
    # here no need to use outputs.argmax as criterion CrossEntropyLoss do this and softmax on our behalf
    loss = criterion(outputs, labels.long()) 
    
    loss.backward()
    optimizer.step()
    if (epoch+1) % 50 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    

Epoch [50/200], Loss: 0.2218
Epoch [100/200], Loss: 0.1097
Epoch [150/200], Loss: 0.1078
Epoch [200/200], Loss: 0.1070


In [44]:
def evaluationFunction():
    model.eval()
    generatedString = ""
    radomStartingPoint = np.random.randint(low=0,high=inputs.shape[0])
    modelInput = inputs[radomStartingPoint]
    startingInteger = torch.argmax(modelInput, dim=0).item()
    startingCharacter = int_to_char[startingInteger]
    #print(radomStartingPoint,modelInput)
    for i in range(50):
        output = model(modelInput)
        predicted_index = torch.argmax(output, dim=0).item()
        char = int_to_char[predicted_index]
        generatedString += char
        modelInput = torch.Tensor(np.eye(vocabSize)[char_to_int[char]])
    
    print(f"Starting Character Randomly choosen in Integer: {startingInteger} in Alphabet: {startingCharacter}")
    print("Generated String:", generatedString)

In [45]:
evaluationFunction()

Starting Character Randomly choosen in Integer: 3 in Alphabet: b
Generated String: cde abcde abcde abcde abcde abcde abcde abcde abcd


### Text Generation with Input as Sequence

In [46]:
dataMatrix , vocabSize, char_to_int,int_to_char = fileHandler("C:/Users/Lenovo/Desktop/My Git Repo/abcde_edcba.txt")

In [47]:
# Prepare training input-output pairs
X_train = np.concatenate((dataMatrix[:-2], dataMatrix[1:-1]), axis=1)  # Concatenate two consecutive one-hot-encoded vectors along columns
Y_train = dataMatrix[2:]  # Next three one-hot-encoded vectors as output

inputs = torch.Tensor(X_train)
#Take the max number (1 in our case) indice of the one hot enocding alongside column
labels = torch.Tensor(np.argmax(Y_train, axis=1)) 

In [229]:
print(dataMatrix[0],dataMatrix[1],dataMatrix[2])

[0. 0. 1. 0. 0. 0. 0.] [0. 0. 0. 1. 0. 0. 0.] [0. 0. 0. 0. 1. 0. 0.]


In [230]:
print(X_train[0],Y_train[0],labels[0].item())

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.] [0. 0. 0. 0. 1. 0. 0.] 4.0


In [48]:
model  = TextGeneratorMLP(vocabSize*2,vocabSize)

In [49]:
print(model)

TextGeneratorMLP(
  (InputLayer): Linear(in_features=14, out_features=128, bias=True)
  (HiddenLayer): Linear(in_features=128, out_features=256, bias=True)
  (HiddenLayer2): Linear(in_features=256, out_features=64, bias=True)
  (OutputLayer): Linear(in_features=64, out_features=7, bias=True)
)


In [50]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [51]:
num_epochs = 150
for epoch in range(num_epochs):
    model.train()
    
    optimizer.zero_grad()
    outputs = model(inputs)
    # here no need to use outputs.argmax as criterion CrossEntropyLoss do this and softmax on our behalf
    loss = criterion(outputs, labels.long()) 
    
    loss.backward()
    optimizer.step()
    if (epoch+1) % 50 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [50/150], Loss: 0.0860
Epoch [100/150], Loss: 0.0025
Epoch [150/150], Loss: 0.0013


In [58]:
def evaluationFunction():
    model.eval()
    generatedString = ""
    radomStartingPoint = np.random.randint(low=0,high=inputs.shape[0])
    modelInput = inputs[radomStartingPoint]
    startingInteger1 = torch.argmax(modelInput[:7], dim=0).item()
    startingInteger2 = torch.argmax(modelInput[7:], dim=0).item()
    startingCharacter1 = int_to_char[startingInteger1]
    startingCharacter2 = int_to_char[startingInteger2]
    #print(radomStartingPoint,modelInput)
    for i in range(50):
        #print(modelInput)
        output = model(modelInput)
        predicted_index = torch.argmax(output, dim=0).item()
        char = int_to_char[predicted_index]
        generatedString += char
        newCharacterEncoding = torch.Tensor(np.eye(vocabSize)[char_to_int[char]])
        modelInput = torch.cat((modelInput[-7:],newCharacterEncoding),dim=0)

    print(f"Starting Characters Randomly choosen in Alphabets: {startingCharacter1,startingCharacter2}")
    print("Generated String:", generatedString)

In [63]:
evaluationFunction()

Starting Characters Randomly choosen in Alphabets: ('d', 'e')
Generated String:  edcba
abcde edcba
abcde edcba
abcde edcba
abcde e


In [60]:
inputs.shape[0]

153598

### Text Generation with MLP Hidden State Implementation

#### Preparing Data

In [3]:
dataMatrix , vocabSize, char_to_int,int_to_char = fileHandler("C:/Users/Lenovo/Desktop/My Git Repo/abcde.txt")

In [4]:
# Prepare training input-output pairs
X_train = dataMatrix[:-1]  # All rows except the last one
Y_train = dataMatrix[1:]   # All rows except the first one

inputs = torch.Tensor(X_train)
#Take the max number (1 in our case) indice of the one hot enocding alongside column
labels = torch.Tensor(np.argmax(Y_train, axis=1)) 

#### Implementing Model

In [115]:
class TextGeneratorHiddenStateMLP(nn.Module):
    def __init__(self,inputSize,outputSize,hiddenStateSize):
        super().__init__()
        
        self.hiddenStateSize = hiddenStateSize
        self.InputLayer = nn.Linear(in_features=inputSize+hiddenStateSize,out_features=128)
        self.HiddenLayer = nn.Linear(in_features=128,out_features=hiddenStateSize)
        self.OutputLayer = nn.Linear(in_features=hiddenStateSize,out_features=outputSize)
    
    def forward(self,X,hiddenState=None):
        if hiddenState is None:
            hiddenState = torch.zeros(self.hiddenStateSize,requires_grad=False) # 1 row and hiddenStateSize columns
        
        outputs = []
        for xPoints in X:
            prevHiddenState = hiddenState
            networkInput = torch.concat((xPoints,prevHiddenState),dim=0)
            hiddenState = F.relu(self.HiddenLayer(F.relu(self.InputLayer(networkInput))))
            yt = self.OutputLayer(hiddenState)
            outputs.append(yt)
        
        return torch.stack(outputs), hiddenState

In [116]:
hiddenStateSize=10
model = TextGeneratorHiddenStateMLP(vocabSize,vocabSize,hiddenStateSize)
model

TextGeneratorHiddenStateMLP(
  (InputLayer): Linear(in_features=17, out_features=128, bias=True)
  (HiddenLayer): Linear(in_features=128, out_features=10, bias=True)
  (OutputLayer): Linear(in_features=10, out_features=7, bias=True)
)

In [180]:
#Remember you can change parameters while training by interuppting the kernel and running it again, it will start from where it stopped.  
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.10)

#### Training The Model

In [181]:
num_epochs = 20
prevLoss = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs,_ = model(inputs) #As in training we don't need hidden state 
    

    # here no need to use outputs.argmax as criterion CrossEntropyLoss do this and softmax on our behalf
    loss = criterion(outputs, labels.long())
    if loss.item() > prevLoss:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Now breaking loop')
        break
    else:
        prevLoss = loss.item()
    loss.backward()
    optimizer.step()
    
    '''Remember you can change evaluation functions and this loop logic while training by interuppting the kernel 
    and running it again, it will start from where it stopped.  '''
    if ((epoch+1) % 2 == 0 ) or epoch==0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        evaluationFunction1()
        evaluationFunction2()

Epoch [1/20], Loss: 0.3567
Starting Character Randomly choosen in Integer: 0 in Alphabet: 

Generated String: ababababababababababababababababababababababababab
Starting Character Randomly choosen in Integer: [5, 6, 1, 2] in Alphabet: ['d', 'e', ' ', 'a']
Generated Character: b
Epoch [2/20], Loss: 13.5137, Now breaking loop


In [135]:
def evaluationFunction1():
    model.eval()
    generatedString = ""
    radomStartingPoint = np.random.randint(low=0,high=inputs.shape[0])
    modelInput = inputs[radomStartingPoint]
    startingInteger = torch.argmax(modelInput, dim=0).item()
    startingCharacter = int_to_char[startingInteger]
    #print(radomStartingPoint,modelInput)
    prevHiddenState=None
    for i in range(50):
        output,prevHiddenState = model(modelInput.reshape(1,-1),prevHiddenState)
        predicted_index = torch.argmax(output, dim=1).item()
        char = int_to_char[predicted_index]
        generatedString += char
        modelInput = torch.Tensor(np.eye(vocabSize)[char_to_int[char]])
    
    print(f"Starting Character Randomly choosen in Integer: {startingInteger} in Alphabet: {startingCharacter}")
    print("Generated String:", generatedString)

In [156]:
evaluationFunction1()

Starting Character Randomly choosen in Integer: 4 in Alphabet: c
Generated String: de abcde abcde abcde abcde abcde abcde abcde abcde


In [175]:
def evaluationFunction2():
    model.eval()
    startingInteger = []
    startingCharacter = []
    radomStartingPoint = np.random.randint(low=0,high=inputs.shape[0]-10)
    
    #let's give model a sequence to see what it generates next as a character 
    modelInput = inputs[radomStartingPoint:radomStartingPoint+4,:]    
    for characterHotEncode in modelInput:
        integer = torch.argmax(characterHotEncode, dim=0).item()
        startingInteger.append(integer)
        startingCharacter.append(int_to_char[integer])

    output,prevHiddenState = model(modelInput)
    lastOutput = output[-1] #As our concern is with last output which will tell what comes after running through sequence
    predicted_index = torch.argmax(lastOutput, dim=0).item()
    char = int_to_char[predicted_index]
    
    print(f"Starting Character Randomly choosen in Integer: {startingInteger} in Alphabet: {startingCharacter}")
    print("Generated Character:", char)

In [176]:
evaluationFunction2()

Starting Character Randomly choosen in Integer: [6, 1, 2, 3] in Alphabet: ['e', ' ', 'a', 'b']
Generated Character: c
